A fast food restaurant chain needs to ship foods from two suppliers (node 1 and 2 ) to the distribution centers (node 5 and 6). Shipments can be made directly or through 2 warehouses (node 3 and 4). At most 100 tons can be shipped between any two nodes. Supplier capacities are 160 tons and 200 tons respectively and distribution center demands are 180 tons each. Shipping cost tons of each route are given in the table below. 



|      | To |    |    |    |
|------|----|----|----|----|
| From |  3 |  4 |  5 |  6 |
|   1  |  8 | 13 | 25 | 28 |
|   2  | 15 | 12 | 26 | 25 |
|   3  |    |  6 | 16 | 17 |
|   4  |  6 |    | 14 | 16 |

In [26]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from pulp import *

Model initialization

In [28]:
model = LpProblem("Transhippment_Problem",LpMinimize)

**Formulating the problem**

**Supply nodes** : 1 and 2                             
**Intermedite/Transient nodes** : 3 and 4                         
**Destination nodes** : 5 and 6

In [29]:
supply_nodes = [i for i in range(1, 5)]
destination_nodes = [i for i in range (3, 7)]

In [30]:
shipment_route = [(i, j) for i in supply_nodes for j in destination_nodes]

In [31]:
route_decision_var = LpVariable.dicts("route_decision", shipment_route, 0, None, "Integer")

In [32]:
route_available = [1, 1, 1, 1,
                   1, 1, 1, 1,
                   0, 1, 1, 1,
                   1, 0, 1, 1]

route_cost = [8, 13, 25, 28,
              15, 12, 26, 25,
              0, 6, 16, 17,
              6, 0, 14, 16]

supplier_capacities = [160, 200]

destination_demand = [180, 180]

In [33]:
availability_mapped = dict(zip(shipment_route, route_available))
cost_mapped = dict(zip(shipment_route, route_cost))
capacities_mapped = dict(zip(list(set(supply_nodes) - set(destination_nodes)), supplier_capacities))
demand_mapped = dict(zip(list(set(destination_nodes) - set(supply_nodes)), destination_demand))

Defining the Objective Function

In [34]:
model += lpSum(route_decision_var[(i, j)] * cost_mapped[(i, j)] for i in supply_nodes for j in destination_nodes)

Adding the constraints

1. Supply from the supply nodes <= Supplier capacities

In [35]:
for i in supply_nodes:
  if i not in destination_nodes:
    model+= lpSum(route_decision_var[(i, j)]* availability_mapped[(i, j)] for j in destination_nodes) <= capacities_mapped[i]

2. Supply to the final destination nodes >= Destination Demand

In [36]:
for j in destination_nodes:
  if j not in supply_nodes:
    model+= lpSum(route_decision_var[(i, j)]* availability_mapped[(i, j)] for i in supply_nodes) >= demand_mapped[j]

3. At Intermediate nodes:                    
     Inflow = Outflow

In [37]:
for i in supply_nodes:
  if i in destination_nodes:
    model+= lpSum(route_decision_var[(j, i)] for j in supply_nodes) == lpSum(route_decision_var[(i, j)] for j in destination_nodes)

4. Each route capacity = 100

In [38]:
for i in supply_nodes:
  for j in destination_nodes:
    model += route_decision_var[(i, j)] <= 100

Solution and Interpretation

In [39]:
model.solve

<bound method LpProblem.solve of Transhippment_Problem:
MINIMIZE
8*route_decision_(1,_3) + 13*route_decision_(1,_4) + 25*route_decision_(1,_5) + 28*route_decision_(1,_6) + 15*route_decision_(2,_3) + 12*route_decision_(2,_4) + 26*route_decision_(2,_5) + 25*route_decision_(2,_6) + 6*route_decision_(3,_4) + 16*route_decision_(3,_5) + 17*route_decision_(3,_6) + 6*route_decision_(4,_3) + 14*route_decision_(4,_5) + 16*route_decision_(4,_6) + 0
SUBJECT TO
_C1: route_decision_(1,_3) + route_decision_(1,_4) + route_decision_(1,_5)
 + route_decision_(1,_6) <= 160

_C2: route_decision_(2,_3) + route_decision_(2,_4) + route_decision_(2,_5)
 + route_decision_(2,_6) <= 200

_C3: route_decision_(1,_5) + route_decision_(2,_5) + route_decision_(3,_5)
 + route_decision_(4,_5) >= 180

_C4: route_decision_(1,_6) + route_decision_(2,_6) + route_decision_(3,_6)
 + route_decision_(4,_6) >= 180

_C5: route_decision_(1,_3) + route_decision_(2,_3) + 0 route_decision_(3,_3)
 - route_decision_(3,_4) - route_decis

In [40]:
model.solve()

1

In [41]:
value(model.objective)

9080.0

In [42]:
for i in route_decision_var:
  print(f"{i} qty {route_decision_var[i].varValue}")

(1, 3) qty 100.0
(1, 4) qty 0.0
(1, 5) qty 60.0
(1, 6) qty 0.0
(2, 3) qty 0.0
(2, 4) qty 100.0
(2, 5) qty 0.0
(2, 6) qty 100.0
(3, 3) qty 0.0
(3, 4) qty 0.0
(3, 5) qty 20.0
(3, 6) qty 80.0
(4, 3) qty 0.0
(4, 4) qty 0.0
(4, 5) qty 100.0
(4, 6) qty 0.0
